# Heat transfer on skin

* Model the heat exchange of a human with the environment
* Use the heat exchange of a bare forearm in an room environment as an basic example
* Consider heat radiation, free and forced convection
* Neglect evaporation

## Boundary conditions

In [1]:
from thermca import *

# Units as SI, temperatures as °C
room_air_temperature = 22
room_surface_temperature = 22
forearm_temperature = 36
emission_coefficient_room_surfaces = .8
emission_coefficient_skin = .95
room_air_velocity = .1  # Rough estimation for slightly occupied rooms
forearm_radius = .035
forearm_length = .25

## Radiation

Calculate the radiation dependent on temperature, orientation and characteristics
of the arm surface and the surfaces of the environment around the arm. Start with 
the resulting emissivity of two surfaces in thermal radiation exchange. This is 
needed as an input to calculate the heat transfer coefficient (film coefficient).

In [2]:
resulting_emissivity = radiation.res_emis(
    emis0=emission_coefficient_skin,
    emis1=emission_coefficient_room_surfaces, 
    view0=1, # The skin surface sees the room surface completely
    view1=1, # The room surface sees the skin surface completely
)
print(f"{resulting_emissivity=}")

resulting_emissivity=0.7676767676767677


### Calculate the radiation heat transfer coefficient

The library function `radiation.therm_radn` is available for determining the radiation 
heat transfer coefficient.

The library functions work in two steps. First, they are parameterised by the "additional" arguments starting at argument number 4. The library functions return a "parameterised function" that can be consumed by model elements. The second step is to call that function. In the case of a simulation, the parametrised function is given to the model elements and called 
internally by the solver. Otherwise it must be called manually, which is shown here.

The parameters of this parametrised function are standardised. They are the temperatures 
of the elements in heat exchange and optionally the material influencing the heat transfer.
In the case of radiation, the material parameter is irrelevant.

In [3]:
parametrised_radiation_function = radiation.therm_radn(res_emis=resulting_emissivity)
radiation_heat_coefficient = parametrised_radiation_function(
    surf0_temp=forearm_temperature, 
    surf1_temp=room_surface_temperature,
)
print(f"{radiation_heat_coefficient=} W/(m²K)")

radiation_heat_coefficient=4.8055738452682135 W/(m²K)


## Convection

### Calculate the heat transfer coefficient of free convection 

The convection heat transfer on a hanging forearm is similar to convection on a vertical cylinder. This in turn is similar to the convection on a vertical surface which is used in this case.

The vertical surface convection model is provided by the `free_conv.vert_surf` library function.

In [4]:
parametrised_convection_fuction = free_conv.vert_surf(surf_hgt=forearm_length)
convection_coefficient = parametrised_convection_fuction(forearm_temperature, room_air_temperature, fluids.air)
print(f"{convection_coefficient=} W/(m²K)")

convection_coefficient=3.889609859462753 W/(m²K)


### Calculate the heat transfer coefficient of forced convection 

In occupied rooms, forced air flows are generated by a variety of processes. We assume that the forearm is surrounded by a light flow of air in a right-angled direction. This is approximated by the convection model provided by the `forced_conv.cyl_cross_flow` library function.

In [5]:
parametrised_forced_convection_fuction = forced_conv.cyl_cross_flow(
    vel=room_air_velocity, 
    rad=forearm_radius,
)
forced_convection_coefficient = parametrised_forced_convection_fuction(
    surf_temp=forearm_temperature, 
    fluid_temp=room_air_temperature, 
    fluid=fluids.air,
)
print(f"{forced_convection_coefficient=} W/(m²K)")

forced_convection_coefficient=3.8022233532323804 W/(m²K)


### Combined acting of forced and natural convection

A common approach to describe combined acting of forced and natural convection is provided by the `combd_film.mix_conv` library function. It distinguishes between assisting, opposing and transverse convection flows. 

Parametrised functions as well as scalars can be passed to this function.

In [6]:
parametrised_convection_fuction = combd_film.mix_conv(
    parametrised_convection_fuction, 
    parametrised_forced_convection_fuction, 
    flow=TRANSVERSE,
)
combined_convection_coefficient = parametrised_convection_fuction(
    forearm_temperature, 
    room_air_temperature, 
    fluids.air,
)
print(f"{combined_convection_coefficient=} W/(m²K)")

combined_convection_coefficient=4.574476631734567 W/(m²K)


## Heat flux

In this example, slightly more heat is released from the skin via radiation than via convection.

In [7]:
radiation_heat_flux = radiation_heat_coefficient * (forearm_temperature - room_surface_temperature)
convection_heat_flux = combined_convection_coefficient * (forearm_temperature - room_air_temperature)
print(f"{radiation_heat_flux=} W/m²\n{convection_heat_flux=} W/m²")

radiation_heat_flux=67.27803383375499 W/m²
convection_heat_flux=64.04267284428393 W/m²
